# 2026 The Best Ski Trip Prediction

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

## Data Parsing and Cleaning

In [ ]:
# Read the excel data
data_fp = list(Path().rglob('../data/2025*.xlsx'))[0]
excel_sheets = pd.read_excel(data_fp, sheet_name=None)
# Unpack the excel sheets
info_data, visitation_data, climate_data = [v for _, v in excel_sheets.items()]

### Parsing the info sheet 
1. Station ID and location
2. Winter ski weeks and dates

In [102]:
location_id = info_data.iloc[21:26,1].copy()
location_id = pd.DataFrame(location_id.str.split(" - ").to_list(), columns=["station_id", "location"])
location_id = location_id.astype({"station_id": str})

print(location_id.info())

location_id.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   station_id  5 non-null      object
 1   location    5 non-null      object
dtypes: object(2)
memory usage: 212.0+ bytes
None


,station_id,location
0,71032,Thredbo AWS
1,71075,Perisher AWS
2,72161,Cabramurra SMHEA AWS
3,83024,Mount Buller
4,83084,Falls Creek


In [ ]:
ski_season_dates = (info_data.iloc[35:, 1:].copy()
                    .rename(columns={"Unnamed: 1": "week", "Unnamed: 2": "dates"})
                    .reset_index(drop=True))
ski_season_dates["dates"] = ski_season_dates["dates"].dt.strftime("%d-%b")

print(ski_season_dates.info())

# Check the weeks
ski_season_dates.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   week    15 non-null     object
 1   dates   15 non-null     object
dtypes: object(2)
memory usage: 372.0+ bytes
None


,week,dates
0,Week 1,09-Jun
1,Week 2,16-Jun
2,Week 3,23-Jun
3,Week 4,30-Jun
4,Week 5,07-Jul


### Parsing the climate data

In [60]:
# Check unclean data
climate_data.tail()

,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
39808,72161,2025,7,24,3.0,-2.7,4.2
39809,72161,2025,7,25,6.0,-2.4,0.2
39810,72161,2025,7,26,2.7,0.0,10.2
39811,72161,2025,7,27,2.5,1.3,15.6
39812,72161,2025,7,28,2.2,-0.1,54.4


In [98]:
# Renaming columns using dictionary
cd_col_names = [col.lower() for col in climate_data.columns]
cd_col_names[0] = "station_id"
cd_col_names[-3:] = ["max_temp_c", "min_temp_c", "rainfall_mm"]
cd_col_names = {col_old: col_new for col_old, col_new in zip(climate_data.columns, cd_col_names)}
climate_data_clean = climate_data.copy().rename(columns=cd_col_names)

# Type formatting
climate_data_clean = climate_data_clean.astype({"station_id": str, "year": str})

# Check datatype
print(climate_data_clean.info())

# Check records
climate_data_clean.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39813 entries, 0 to 39812
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   station_id   39813 non-null  object 
 1   year         39813 non-null  object 
 2   month        39813 non-null  int64  
 3   day          39813 non-null  int64  
 4   max_temp_c   38275 non-null  float64
 5   min_temp_c   38280 non-null  float64
 6   rainfall_mm  37857 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 2.1+ MB
None


,station_id,year,month,day,max_temp_c,min_temp_c,rainfall_mm
39808,72161,2025,7,24,3.0,-2.7,4.2
39809,72161,2025,7,25,6.0,-2.4,0.2
39810,72161,2025,7,26,2.7,0.0,10.2
39811,72161,2025,7,27,2.5,1.3,15.6
39812,72161,2025,7,28,2.2,-0.1,54.4


### Parsing the visitation data

In [93]:
# Converting to long format for easier processing
visitation_data_long = (visitation_data.copy()
                        .melt(id_vars=("Year", "Week"), var_name="location", value_name="visitors")
                        .rename(columns={"Year" : "year", "Week" : "week"}))
visitation_data_long = visitation_data_long.astype({"year": str, "week": str})
visitation_data_long.head()

,year,week,location,visitors
0,2014,1,Mt. Baw Baw,555
1,2014,2,Mt. Baw Baw,804
2,2014,3,Mt. Baw Baw,993
3,2014,4,Mt. Baw Baw,2976
4,2014,5,Mt. Baw Baw,11112


In [94]:
visitation_data_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1485 entries, 0 to 1484
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      1485 non-null   object
 1   week      1485 non-null   object
 2   location  1485 non-null   object
 3   visitors  1485 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 46.5+ KB


## Exporting the data

In [69]:
location_id.to_csv(data_fp.parent / "location_id.csv", index=False)
location_id.to_parquet(data_fp.parent / "location_id.parquet", engine="fastparquet")

In [70]:
ski_season_dates.to_csv(data_fp.parent / "ski_season_dates.csv", index=False)
ski_season_dates.to_parquet(data_fp.parent / "ski_season_dates.parquet", engine="fastparquet")

In [71]:
climate_data_clean.to_csv(data_fp.parent / "climate_data.csv", index=False)
climate_data_clean.to_parquet(data_fp.parent / "climate_data.parquet", engine="fastparquet")

In [ ]:
visitation_data.to_csv(data_fp.parent / "visitation_data_raw.csv", index=False)
visitation_data_long.to_csv